In [1]:
# Import Libaries and Tools
import os
import sys
from astropy.io import fits
import sunpy.map
from aiapy.calibrate import register, update_pointing
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import imageio
import glob

# ACWE utilities
# Root directory of the project
ROOT_DIR = os.path.abspath("../../../../")

# Import ACWE Tools
sys.path.append(ROOT_DIR)
from ACWE_python_fall_2023 import acweSaveSeg_v5, acweRestoreScale
from DatasetTools import DataManagmentTools as dmt

plt.rcParams.update({'image.interpolation': 'None'})

In [ ]:
# Dataset

# Carrington Rotation
CR = 'CR2133'

# Dataset
dataset = '/home/jgra/Coronal Holes/newDataset/'

# ACWE Folders
SaveFolderMag = '/mnt/coronal_holes/Paper 2/Code 02 Observations/'
SaveFolderMag = SaveFolderMag + 'HMI_Experiments/TestEvolutionMethods/Scaled/'
SaveFolderEUV = '/mnt/coronal_holes/Paper 1/Code Paper I Observations/Scaled/'

# ACWE Data
magMethod  = 'Uni' # 'FluxImb' 'Homogentiy' 'Skew' 
ACWEprefix = 'ACWEresize_param1.'
acweChoice = '193'

# Comparison data
suffix = '.MagToEUV.npz'

# Number of samples
I = 50

In [ ]:
# Open File and Prepare for ACWE
def openAIA(filename):
    
    # Extract Image and Header Data
    hdulist = fits.open(filename)
    hdulist.verify('silentfix') # no clue why this is needed for successful data read
    h = hdulist[1].header
    J = hdulist[1].data
    hdulist.close()
    
    # Update to Level 1.5 Data Product
    if h['LVL_NUM'] < 1.5:
        m = sunpy.map.Map((J,h))    # Create Sunpy Map
        m = update_pointing(m)      # Update Header based on Latest Information
        m_registrered = register(m) # Recenter and rotate to Solar North
        I = m_registrered.data
        # Undo Keword Renaming
        H = dict()
        for k in m_registrered.meta.keys(): 
            H[k.upper()] = m_registrered.meta[k] 
    # Skip if already Level 1.5
    else:
        I = J*1
        H = dict(h)
        
        # Prepare Display Version
    Idsp = np.clip(I,20,2500)
    Idsp = np.log10(Idsp)
    Idsp = Idsp - np.min(Idsp)
    Idsp = Idsp/np.max(Idsp)
    
    return I,Idsp,H

In [ ]:
# Create or find save location
crSaveFolder1 = SaveFolderMag  + magMethod + '/' + CR + '/'
crSaveFolder2 = SaveFolderEUV  + CR + '/'
datasetFolder = dataset + CR + '/'

# Get Dataset
imageFiles  = sorted(glob.glob(datasetFolder + '*/*.' + acweChoice + '.*'))
segMagFiles = sorted(glob.glob(crSaveFolder1 + '*/' + ACWEprefix + '*.'+ acweChoice + '.*'))
segEUVfiles = sorted(glob.glob(crSaveFolder2 + '*/' + ACWEprefix + '*.'+ acweChoice + '.*'))

In [ ]:
# Collection of Results
datasetfile = glob.glob('Results/' + CR + '*.' + magMethod + suffix)[0]

# open and retreave data
data = np.load(datasetfile, allow_pickle=True)
lst = data.files

IOU  = data[lst[0]]
SSIM = data[lst[1]]
GCE  = data[lst[2]]
LCE  = data[lst[3]]

In [ ]:
# Ensure Folder Exists
saveDirectory = '/Figures/TestEvolutionMethods/' + magMethod + '/LowestIOU/' + CR + '/'
saveDirectory = ROOT_DIR + saveDirectory

if not os.path.exists(saveDirectory):
    os.makedirs(saveDirectory)

In [ ]:
# Get List of IOU
IOUordered = np.sort(IOU)

for i in range(I):
    
    # find Index of desired Segmentation
    j = np.where(IOU == IOUordered[i])[0][0]
    
    # Get File Names
    imageFile  = imageFiles[j]
    segMagFile = segMagFiles[j]
    segEUVfile = segEUVfiles[j]
    
    # Open EUV image
    _,Idsp,H = openAIA(imageFile)
    
    # Open Segmentations
    _,MAH,MagSeg = acweSaveSeg_v5.openSeg(segMagFile)
    _,EAH,EUVseg = acweSaveSeg_v5.openSeg(segEUVfile)
    
    print('\nIOU: ',IOU[j])
    
    # Show Resutls
    plt.figure(figsize=[5,2],dpi=300)
    plt.rcParams.update({'font.size': 8})
    title = H['T_REC']
    plt.suptitle(title)
    plt.rcParams.update({'font.size': 5})
    plt.subplot(1,3,1)
    plt.imshow(np.flip(Idsp,axis=0),cmap='gray')
    plt.axis(False)
    plt.title('EUV Observation')
    plt.subplot(1,3,2)
    plt.imshow(np.flip(EUVseg,axis=0),interpolation='None')
    plt.axis(False)
    plt.title('ACWE Segmentation')
    plt.subplot(1,3,3)
    plt.imshow(np.flip(MagSeg,axis=0),interpolation='None')
    plt.axis(False)
    plt.title('QUACK Segmentation')
    title = saveDirectory + title.replace(':','') + 'QUACK.png'
    plt.savefig(title)
    plt.show()
    
    plt.subplot(1,2,1)
    plt.imshow(np.flip(EAH['INIT_MASK'],axis=0),interpolation='None')
    plt.title('Seed - ACWE')
    plt.subplot(1,2,2)
    plt.imshow(np.flip(MAH['INIT_MASK'],axis=0),interpolation='None')
    plt.title('Seed - QUACK')
    plt.show()
    print()
    print()